### Import module

In [53]:
import numpy as np
from ultralytics import YOLO
import cv2

In [54]:
class PushUpCounter:
    def __init__(self, down_threshold=80, up_threshold=100, buffer_size=3, min_confidence=0.5):
        self.down_threshold = down_threshold
        self.up_threshold = up_threshold
        self.buffer_size = buffer_size
        self.min_confidence = min_confidence
        
        # State variables
        self.current_state = "unknown"
        self.selected_arm = None
        self.angle_buffer = []
        self.count = 0
    
    def calculate_angle(self, shoulder_point, elbow_point, wrist_point):
        shoulder = np.array(shoulder_point)
        elbow = np.array(elbow_point)
        wrist = np.array(wrist_point)
        
        elbow_to_shoulder_vector = shoulder - elbow
        elbow_to_wrist_vector = wrist - elbow

        cos_angle = np.dot(elbow_to_shoulder_vector, elbow_to_wrist_vector) / (np.linalg.norm(elbow_to_shoulder_vector) * np.linalg.norm(elbow_to_wrist_vector))
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        angle_degrees = np.degrees(angle)
        return angle_degrees
        
    def process_frame(self, results):
       
        if not results or not results[0].keypoints:
            return {
                'count': self.count,
                'state': self.current_state,
                'angle': None,
                'arm': 'NONE',
                'message': 'No keypoints detected'
            }
        
        confidences = results[0].keypoints.conf[0]
        keypoints = results[0].keypoints.xy[0]
        
        left_angle_hand = float('inf')
        right_angle_hand = float('inf')
        
        # Calculate left arm angle
        if len(keypoints) > 9:
            confidence_left_average = (confidences[7] + confidences[5] + confidences[9]) / 3
            if confidence_left_average > self.min_confidence:
                left_elbow_point = keypoints[7]
                left_shoulder_point = keypoints[5]
                left_wrist_point = keypoints[9]
                left_angle_hand = self.calculate_angle(left_shoulder_point, left_elbow_point, left_wrist_point)
        
        # Calculate right arm angle
        if len(keypoints) > 10:
            confidence_right_average = (confidences[8] + confidences[6] + confidences[10]) / 3
            if confidence_right_average > self.min_confidence:
                right_elbow_point = keypoints[8]
                right_shoulder_point = keypoints[6]
                right_wrist_point = keypoints[10]
                right_angle_hand = self.calculate_angle(right_shoulder_point, right_elbow_point, right_wrist_point)
        
        # Arm selection logic - choose once and stick with it
        if self.selected_arm is None:
            if (confidence_left_average > confidence_right_average and 
                left_angle_hand != float('inf')):
                self.selected_arm = "left"
            elif right_angle_hand != float('inf'):
                self.selected_arm = "right"
        
        # Get current angle from selected arm
        if self.selected_arm == "left" and left_angle_hand != float('inf'):
            raw_angle = left_angle_hand
            arm_label = "LEFT"
        elif self.selected_arm == "right" and right_angle_hand != float('inf'):
            raw_angle = right_angle_hand
            arm_label = "RIGHT"
        else:
            return {
                'count': self.count,
                'state': self.current_state,
                'angle': None,
                'arm': 'NONE',
                'message': 'Selected arm not available'
            }
        
        # Apply angle smoothing
        self.angle_buffer.append(raw_angle)
        if len(self.angle_buffer) > self.buffer_size:
            self.angle_buffer.pop(0)
        
        # Use smoothed angle (average of recent frames)
        smoothed_angle = sum(self.angle_buffer) / len(self.angle_buffer)
        
        # State machine with buffer zone (hysteresis)
        previous_state = self.current_state
        if smoothed_angle < self.down_threshold:
            # Clearly in down position
            if self.current_state == "up":
                # Transition from up to down - complete push-up
                self.count += 1
            self.current_state = "down"
        elif smoothed_angle > self.up_threshold:
            # Clearly in up position
            self.current_state = "up"
        # If angle is between thresholds, maintain current state (buffer zone)
        
        return {
            'count': self.count,
            'state': self.current_state,
            'previous_state': previous_state,
            'raw_angle': raw_angle,
            'smoothed_angle': smoothed_angle,
            'arm': arm_label
        }
    
    def reset(self):
        """Reset counter state"""
        self.current_state = "unknown"
        self.selected_arm = None
        self.angle_buffer = []
        self.count = 0

### Test 

In [ ]:
# Example usage of PushUpCounter function
model = YOLO('yolo11n-pose.pt')

# Initialize the push-up counter
pushup_counter = PushUpCounter(
    down_threshold=80,    # < 80 degrees = down state
    up_threshold=100,     # > 100 degrees = up state
    buffer_size=3,        # Average of 3 frames for smoothing
    min_confidence=0.5    # Minimum keypoint confidence
)

# Open video file
cap = cv2.VideoCapture('push.mp4')
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        break
    
    # Predict on current frame
    results = model.predict(source=frame, show=False, verbose=False)
    
    # Process frame with our function
    result_info = pushup_counter.process_frame(results)

    # Display frame
    if results and results[0].keypoints is not None:
        annotated_frame = results[0].plot()
    else:
        annotated_frame = frame
    
    cv2.imshow('Push-up Counter', annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print('Total Frames Processed:', frame_count)
cap.release()
cv2.destroyAllWindows()
print('Total Push-ups:', pushup_counter.count)

/var/folders/7y/tb8vlfv937523rgqdlf0zjy40000gn/T/ipykernel_76560/1611377937.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  shoulder = np.array(shoulder_point)
/var/folders/7y/tb8vlfv937523rgqdlf0zjy40000gn/T/ipykernel_76560/1611377937.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  elbow = np.array(elbow_point)
/var/folders/7y/tb8vlfv937523rgqdlf0zjy40000gn/T/ipykernel_76560/1611377937.py:17: DeprecationWarning: __array__ implementation doesn't accept a copy keyw

Push-up #0 completed at frame: 1 (angle: 122.13)
Push-up #0 completed at frame: 2 (angle: 114.33)
Push-up #0 completed at frame: 3 (angle: 108.73)
Push-up #0 completed at frame: 4 (angle: 97.80)
Push-up #0 completed at frame: 5 (angle: 91.44)
Push-up #0 completed at frame: 6 (angle: 87.60)
Push-up #0 completed at frame: 7 (angle: 86.73)
Push-up #0 completed at frame: 8 (angle: 84.59)
Push-up #0 completed at frame: 9 (angle: 82.34)
Push-up #0 completed at frame: 5 (angle: 91.44)
Push-up #0 completed at frame: 6 (angle: 87.60)
Push-up #0 completed at frame: 7 (angle: 86.73)
Push-up #0 completed at frame: 8 (angle: 84.59)
Push-up #0 completed at frame: 9 (angle: 82.34)
Push-up #1 completed at frame: 10 (angle: 78.61)
Push-up #1 completed at frame: 11 (angle: 74.86)
Push-up #1 completed at frame: 12 (angle: 73.08)
Push-up #1 completed at frame: 13 (angle: 72.61)
Push-up #1 completed at frame: 14 (angle: 74.28)
Push-up #1 completed at frame: 10 (angle: 78.61)
Push-up #1 completed at frame: 